In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import re

pd.set_option('display.max_columns', None)


In [ ]:
data_folder = Path("data/fly_1_14")
timestamps_file = list(data_folder.glob('*timestamps.csv'))[0]
timestamps = pd.read_csv(timestamps_file)
timestamps.columns = timestamps.columns.str.strip().str.replace(' ', '_')
timestamps['time'] = pd.to_datetime(timestamps['timestamp'], unit='ns')
timestamps.set_index('time', inplace=True)
timestamps[::10]

In [ ]:
time = pd.to_datetime('2023-07-11 18:13:25')
idx  = timestamps.index.get_loc(time, method="nearest")
frame_id = timestamps.iloc[idx].frame_id

In [ ]:
list(data_folder.glob('*'))

In [ ]:
wheel_file = list(data_folder.glob('*tracker.csv'))[0]
wheel_df = pd.read_csv(wheel_file, index_col=0)
wheel_df.columns = wheel_df.columns.str.strip().str.replace(' ', '_')
wheel_df.rename(
            columns={
                "absolute_rotation_cam_0": "yaw",
                "absolute_rotation_cam_1": "roll",
            },
            inplace=True,
        )
df = wheel_df.filter(regex='roll|yaw').copy()
# wheel_df.filter(regex='timestamp|position|sequence|heading')

In [ ]:
df.yaw.plot()

In [ ]:
df.loc[df['yaw'].diff().abs() > np.pi, 'yaw']  = np.nan
df.iloc[1000:1500].yaw.plot()

In [ ]:
data_folder = Path("data/rig2_experiment_13")
experiment_number = re.findall(r"nt_(\d+)", str(data_folder))[0]
experiment_number

In [ ]:
wheel_df.filter(regex='timestamp|position|sequence|heading|ute_rotation_cam_[01]')
wheel_df.filter(regex='heading|ute_rotation_cam_[01]').iloc[:500].plot()

In [ ]:
wheel_df

In [ ]:

wheel_df = pd.read_csv(wheel_file)
wheel_df.columns = wheel_df.columns.str.strip().str.replace(" ", "_")
wheel_df = wheel_df.filter(regex="timestamp|absolute_rotation_cam_[01]")
wheel_df.rename(
    columns={"absolute_rotation_cam_0": "yaw", "absolute_rotation_cam_1": "roll"},
    inplace=True,
)
wheel_df["time"] = (
    pd.to_datetime(wheel_df["timestamp"], unit="ns")
    .dt.tz_localize("UTC")
    .dt.tz_convert("US/Eastern")
)
wheel_df.set_index("time", inplace=True)
wheel_df


In [ ]:
import wheel_movie as wm
from importlib import reload

reload(wm)

movie = wm.WheelMovie(
    wheel_data_file=list(data_folder.glob("*tracker.csv"))[0],
    movie_file=list(data_folder.glob("*.avi"))[0],
    movie_timestamp_file=list(data_folder.glob("*timestamps.csv"))[0],
    fly_name = 'Fly 14',
    plot_width = '30s',
    output_path = data_folder,
    output_filename = 'test',
    total_frames = 500,
    show_nth_frame=3,
    fps= 20,
    zero_pos_frac=0.65,
    
)

movie.make_movie_paralell()